In [2]:
import pyspark
sc = pyspark.SparkContext()

In [3]:
sql = pyspark.sql.SQLContext(sc)

In [4]:
rdd = sc.newAPIHadoopRDD("org.elasticsearch.hadoop.mr.EsInputFormat",
                         "org.apache.hadoop.io.NullWritable", 
                         "org.elasticsearch.hadoop.mr.LinkedMapWritable", 
                         conf={"es.resource":"sra",
                               "es.nodes.wan.only": "true",
                               "es.net.http.auth.user": "omicidx",
                               "es.net.http.auth.pass": "Asdf1234%",
                               "es.net.ssl" : "true",
                               "es.nodes": "c0b1083b4d78495d3b29a72d74fcea41.us-east-1.aws.found.io",
                               "es.port": "9243"
                              })

In [3]:
entities = 'experiment run sample study'.split()
sra = {}
for entity in entities:
    print(entity)
    sra[entity] = sql.read.parquet('s3://omicidx.cancerdatasci.org/sra/NCBI_SRA_Mirroring_20180501_Full/parquet/{}_parquet/'.format(entity))
    sra[entity].cache()

experiment
run
sample
study


In [5]:
def prefix_table_name(df, tbl_name):
    if(tbl_name + '_accession' in df.columns):
        df = df.drop(tbl_name + "_accession")
    for c in df.columns:
        if(c.endswith('accession')):
            continue
        df = df.withColumnRenamed(c, tbl_name + "_" + c)
    df = df.withColumnRenamed('accession', tbl_name + "_accession")
    return(df)

In [6]:
sra2 = {}
for entity in sra.keys():
    sra2[entity] = prefix_table_name(sra[entity], entity)

In [8]:
sra2['experiment'].printSchema()

root
 |-- experiment_accession: string (nullable = true)
 |-- experiment_alias: string (nullable = true)
 |-- experiment_attributes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- tag: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- experiment_broker_name: string (nullable = true)
 |-- experiment_center_name: string (nullable = true)
 |-- experiment_description: string (nullable = true)
 |-- experiment_design: string (nullable = true)
 |-- experiment_identifiers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- namespace: string (nullable = true)
 |    |    |-- uuid: string (nullable = true)
 |-- experiment_instrument_model: string (nullable = true)
 |-- experiment_library_construction_protocol: string (nullable = true)
 |-- experiment_library_layout: string (nullable = true)
 |-- experiment_library_layout_length: float (nullable = true)

In [7]:
df = sra2['run'].join(sra2['experiment'], 'experiment_accession', 'left_outer')\
     .join(sra2['sample'], 'sample_accession', 'left_outer')\
     .join(sra2['study'], 'study_accession', 'left_outer')

In [21]:
df = sql.read.parquet('s3://omicidx.cancerdatasci.org/sra/NCBI_SRA_Mirroring_20180401_Full/parquet/study_parquet/')

In [4]:
conf={"es.resource":"sra5",
                               "es.nodes.wan.only": "true",
                               "es.net.http.auth.user": "omicidx",
                               "es.net.http.auth.pass": "Asdf1234%",
                               "es.net.ssl" : "true",
                               "es.nodes": "c0b1083b4d78495d3b29a72d74fcea41.us-east-1.aws.found.io",
                               "es.port": "9243"
                              }


In [6]:
df3 = sql.read.format("org.elasticsearch.spark.sql")\
    .option("es.nodes.wan.only","true")\
    .option("es.net.http.auth.user","omicidx")\
    .option("es.net.http.auth.pass","Asdf1234%")\
    .option("es.net.ssl","true")\
    .option("es.nodes","c0b1083b4d78495d3b29a72d74fcea41.us-east-1.aws.found.io")\
    .option("es.read.field.as.array.include","identifiers,attributes,xrefs")\
    .option("es.mapping.date.rich", "true")\
    .option("es.port","9243").load('sra_study/doc')

In [7]:
df3.show()

+-----------+--------+-----+-------------------+-------------------+-------------------+-------------------+------+--------------------+---------+--------------------+--------------------+-----------+-----------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+
| BioProject|     GEO|Insdc|     LastMetaUpdate|         LastUpdate|          Published|           Received|Status|            abstract|accession|               alias|          attributes|broker_name|center_name|         description|         identifiers|study_accession|          study_type|               title|               xrefs|
+-----------+--------+-----+-------------------+-------------------+-------------------+-------------------+------+--------------------+---------+--------------------+--------------------+-----------+-----------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------

In [9]:
df3.write.format("org.elasticsearch.spark.sql")\
    .option("es.nodes.wan.only","true")\
    .option("es.net.http.auth.user","omicidx")\
    .option("es.net.http.auth.pass","Asdf1234%")\
    .option("es.net.ssl","true")\
    .option("es.nodes","c0b1083b4d78495d3b29a72d74fcea41.us-east-1.aws.found.io")\
    .option("es.port","9243")\
    .option("es.mapping.id", "accession")\
    .save('sra_study2/doc')

In [ ]:
for i in sra.keys():
    sra[i].write.format("org.elasticsearch.spark.sql")\
    .option("es.nodes.wan.only","true")\
    .option("es.net.http.auth.user","omicidx")\
    .option("es.net.http.auth.pass","Asdf1234%")\
    .option("es.net.ssl","true")\
    .option("es.nodes","c0b1083b4d78495d3b29a72d74fcea41.us-east-1.aws.found.io")\
    .option("es.port","9243")\
    .option("es.mapping.id", "{}_accession".format(i))\
    .save('sra_{}/doc'.format(i))

In [13]:
from pyspark.sql.functions import col
accession_info = sql.read.format('csv').options(header=True).options(delimiter="\t")\
    .load("s3://omicidx.cancerdatasci.org/sra/NCBI_SRA_Mirroring_20180501_Full/" + "SRA_Accessions.tab.bz2")\
    .select("Accession", "Spots", "Bases")\
    .filter("Accession like 'DRR%'")

In [14]:
accession_info.show()

+---------+--------+----------+
|Accession|   Spots|     Bases|
+---------+--------+----------+
|DRR000001|10148174| 730668528|
|DRR000002| 8316737| 598805064|
|DRR000003| 6459732| 232550352|
|DRR000004| 6061180| 218202480|
|DRR000005| 6040762| 217467432|
|DRR000006| 5936619| 213718284|
|DRR000007| 3503763| 126135468|
|DRR000363|43900909|1580432724|
|DRR000364|44984884|1619455824|
|DRR000365|45416179|1634982444|
|DRR000366|44948037|1618129332|
|DRR000008|19153889| 689540004|
|DRR000009|17668118| 636052248|
|DRR000010|18079621| 650866356|
|DRR000011|15788579| 568388844|
|DRR000012|15075547| 542719692|
|DRR000013|15490013| 557640468|
|DRR000014|16388392| 589982112|
|DRR000015|15287389| 550346004|
|DRR000016|15445050| 556021800|
+---------+--------+----------+
only showing top 20 rows



In [4]:
sra['run'].show()

+---------+--------------------+-----------+-----------+--------------------+--------------------+------+--------------------+---------+--------------------+--------+-----------+---------+-----------+------+-------------------+-------------------+-------------------+-------------------+----------+-----+-----------+----------+--------------------+-------------------+
|    alias|          attributes|broker_name|center_name|experiment_accession|         identifiers|nreads|               reads|accession|          run_center|run_date|spot_length|    spots|      bases|Status|           Received|          Published|         LastUpdate|     LastMetaUpdate|ReplacedBy|Insdc|file_addons|  FileSize|             FileMd5|           FileDate|
+---------+--------------------+-----------+-----------+--------------------+--------------------+------+--------------------+---------+--------------------+--------+-----------+---------+-----------+------+-------------------+-------------------+---------------

In [5]:
sra['experiment'].show()

+---------+---------+----------+-----------+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-----------------------------+--------------+---------------------+--------------------------+-------------------+--------------------+-----------------+--------------+----------------+-----------+----------------+---------------+--------------------+-----+------+-------------------+-------------------+-------------------+-------------------+----------+-----+
|accession|    alias|attributes|broker_name|center_name|description|              design|experiment_accession|         identifiers|    instrument_model|library_construction_protocol|library_layout|library_layout_length|library_layout_orientation|library_layout_sdev|        library_name|library_selection|library_source|library_strategy|   platform|sample_accession|study_accession|               title|xrefs|Status|           Received|          Published|         LastUpdate|     LastM

In [8]:
df.columns

['study_accession',
 'sample_accession',
 'experiment_accession',
 'run_alias',
 'run_attributes',
 'run_broker_name',
 'run_center_name',
 'run_identifiers',
 'run_nreads',
 'run_reads',
 'run_accession',
 'run_run_center',
 'run_run_date',
 'run_spot_length',
 'run_spots',
 'run_bases',
 'run_Status',
 'run_Received',
 'run_Published',
 'run_LastUpdate',
 'run_LastMetaUpdate',
 'run_ReplacedBy',
 'run_Insdc',
 'run_file_addons',
 'run_FileSize',
 'run_FileMd5',
 'run_FileDate',
 'experiment_alias',
 'experiment_attributes',
 'experiment_broker_name',
 'experiment_center_name',
 'experiment_description',
 'experiment_design',
 'experiment_identifiers',
 'experiment_instrument_model',
 'experiment_library_construction_protocol',
 'experiment_library_layout',
 'experiment_library_layout_length',
 'experiment_library_layout_orientation',
 'experiment_library_layout_sdev',
 'experiment_library_name',
 'experiment_library_selection',
 'experiment_library_source',
 'experiment_library_strat

In [11]:
sra['study'].printSchema()

root
 |-- BioProject: string (nullable = true)
 |-- GEO: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- accession: string (nullable = true)
 |-- alias: string (nullable = true)
 |-- attributes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- tag: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- broker_name: string (nullable = true)
 |-- center_name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- identifiers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- namespace: string (nullable = true)
 |-- study_accession: string (nullable = true)
 |-- study_type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- xrefs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- db: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |-- St